<a href="https://colab.research.google.com/github/DanielOlliveira/projeto_transfer_learning_darknet/blob/main/projeto_transfer_learning_darknet_to_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Notebook Title](http://blog.ibanyez.info/download/B20190408T000000071.jpg)

# How to train YOLOv3 using Darknet on Colab notebook and optimize the VM runtime load times

## Welcome!

This Colab notebook will show you how to:

* Train a **Yolo v3** model using **Darknet** using the Colab **12GB-RAM GPU**.
* Turn Colab notebooks into an effective tool to work on real projects. Dealing with the handicap of a runtime that will **blow up every 12 hours** into the space!
  * Working directly from the files on your computer.
  * Configure your notebook to install everything you need and start training in about a minute (Tested using 550MB dataset).
  * Receive your trained weights directly on your computer during the training. While the notebook is training you can check how it is going using your trained weights in your computer.


#### This notebook is part of the post [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](http://blog.ibanyez.info/blogs/coding/20190410-run-a-google-colab-notebook-to-train-yolov3-using-darknet-in/) I encourage you to visit! You will find a deeper explanation about Google Colab, the goods and the limitations of this great tool.


### These are the steps we'll follow:

* Configure Google Drive and map as network  _Drive_.
* Some utils to help to do some tasks.
* Configure the pre-requisites on the runtime.
  * Check the CUDA installation on the runtime VM.
  * Install cuDNN.
  * Clone and compile Darknet. We'll use a repo based on [AlexeyAB's Darknet repo](https://github.com/AlexeyAB/darknet/). I applied  some changes to make possible to load files from `/My Drive/` and reduced the number of logs on console to speed up the notebook.
  * We'll check that everything works great.
* Explained how to manage your YOLO files on your computer and it will be used transparently from this notebook.

> _**NOTE:** Cells with an annotation **`# Not Necessary cell`**. Can be removed without having any impact. They are only explanatory content._

### Without further ado, let's start!


## STEP 0. Configure runtime to work with GPU

We want to use the **12GB-RAM GPU** hardware acceleration!

Go to **> Menu > Runtime > Configure Runtime Type** And select **GPU** From the **Hardware accelerator** drop down meu

## STEP 1. Connect your files to Google Drive
In order to have your files in your local computer you need to install and configure Google Backup and Sync to keep one folder of your Drive synced with a folder on your computer.

![schema drive.jpg](http://blog.ibanyez.info/download/B20190408T000000060.jpg)

1. Create a folder on your Google Drive named _**darknet**_
2. Configure the Google Backup and Sync as follows. If you don't speak Spanish, you maybe learn some words! **;)**
![Sync Drive.jpg](http://blog.ibanyez.info/download/B20190408T000000063.jpg)

**After this step you'll have a folder called _darknet_ in your local computer. This folder is where you will work with files on your computer**

> _**TIP** - We need to have a good performance downloading data from Drive to Colab. Having a lot of files in your Drive root folder can slow down the things quite a bit. It's a good practice working with Colab to move all your root folder files into a folder_




## STEP 2. Connect the Colab notebook to Google Drive

Now we're gonna map your Google Drive folder. This first step is the only one that will require your manual interaction every time you run your notebook.

* Execute the following cell _(Click on Play button or press CTRL + ENTER)_ and click on the link to authorize your notebook to access to your Google Drive.
* Paste the code Google will give to you and push `enter`

In [ ]:
!pip install -q gdown

# Baixar o arquivo compactado da pasta 'darknet' via link direto do Google Drive
!gdown --id 1FHYRiTgjm4Sqlw93MsdnwblIqCQfBP01 -O darknet.zip

# Extrair o conteúdo do arquivo 'darknet.zip' para o diretório atual (/content)
!unzip -q darknet.zip -d /content/

# Verificar se a pasta foi extraída corretamente
!ls -la /content/darknet

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1FHYRiTgjm4Sqlw93MsdnwblIqCQfBP01
From (redirected): https://drive.google.com/uc?id=1FHYRiTgjm4Sqlw93MsdnwblIqCQfBP01&confirm=t&uuid=3b4cad31-99a3-4205-98fc-4dc4ed25f92b
To: /content/darknet.zip
100% 1.54G/1.54G [00:23<00:00, 65.4MB/s]
total 56
drwxr-xr-x 8 root root 4096 Sep 20 17:57 .
drwxr-xr-x 1 root root 4096 Sep 20 17:57 ..
drwxr-xr-x 2 root root 4096 Sep 20 17:57 annotations
drwxr-xr-x 2 root root 4096 Sep 20 17:57 bin
drwxr-xr-x 2 root root 4096 Sep 20 17:57 cuDNN
-rw-r--r-- 1 root root 9093 Sep  8 06:51 darknet53.conv.74
drwxr-xr-x 2 root root 4096 Sep 20 17:57 dataset_coco_cachorros
drwxr-xr-x 2 root root 4096 Sep 20 17:57 dataset_coco_gatos
drwxr-xr-x 3 root root 4096 Sep 20 17:57 val2017
-rw-r--

Congratulations! Now you can access to your local computer folder directly from here!

Check it out!

In [ ]:
# Liste o conteúdo da pasta do seu ambiente local
!ls -la '/content/darknet'

total 56
drwxr-xr-x 8 root root 4096 Sep 20 17:57 .
drwxr-xr-x 1 root root 4096 Sep 20 17:57 ..
drwxr-xr-x 2 root root 4096 Sep 20 17:57 annotations
drwxr-xr-x 2 root root 4096 Sep 20 17:57 bin
drwxr-xr-x 2 root root 4096 Sep 20 17:57 cuDNN
-rw-r--r-- 1 root root 9093 Sep  8 06:51 darknet53.conv.74
drwxr-xr-x 2 root root 4096 Sep 20 17:57 dataset_coco_cachorros
drwxr-xr-x 2 root root 4096 Sep 20 17:57 dataset_coco_gatos
drwxr-xr-x 3 root root 4096 Sep 20 17:57 val2017
-rw-r--r-- 1 root root 8343 Sep  8 07:06 yolov3.cfg


In [ ]:
!sudo apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (43.5 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 126435 files and directories currently instal

In [ ]:
!tree /content/darknet/

A saída de streaming foi truncada nas últimas 5000 linhas.
│       ├── 000000000724.jpg
│       ├── 000000000776.jpg
│       ├── 000000000785.jpg
│       ├── 000000000802.jpg
│       ├── 000000000872.jpg
│       ├── 000000000885.jpg
│       ├── 000000001000.jpg
│       ├── 000000001268.jpg
│       ├── 000000001296.jpg
│       ├── 000000001353.jpg
│       ├── 000000001425.jpg
│       ├── 000000001490.jpg
│       ├── 000000001503.jpg
│       ├── 000000001532.jpg
│       ├── 000000001584.jpg
│       ├── 000000001675.jpg
│       ├── 000000001761.jpg
│       ├── 000000001818.jpg
│       ├── 000000001993.jpg
│       ├── 000000002006.jpg
│       ├── 000000002149.jpg
│       ├── 000000002153.jpg
│       ├── 000000002157.jpg
│       ├── 000000002261.jpg
│       ├── 000000002299.jpg
│       ├── 000000002431.jpg
│       ├── 000000002473.jpg
│       ├── 000000002532.jpg
│       ├── 000000002587.jpg
│       ├── 000000002592.jpg
│       ├── 000000002685.jpg
│       ├── 000000002923.jpg
│       ├── 0

### UPDATE NOTE

If you want to simplify your paths, you can use a Symbolic link:

`!ln -s "/content/gdrive/My Drive/darknet/" /mydrive`

 Then you'll be able to access your Google Drive files just using `/mydrive` path

**BE CAREFUL: All the paths in this notebook are without using Symbolic link. You'll have to remember to change the path everywhere**

Thanks to **Dennis Kashkin** for this recommendation!

In [ ]:
# Uncomment if you want to use Symbolic link
#!ln -s /content/gdrive/My\ Drive/darknet/ /mydrive
#!ls /mydrive

## STEP 2. Check CUDA release version

Nvidia CUDA is pre-installed on Colab notebooks. Now we'll check the version installed.

> _**BE AWARE:** Maybe some time from the time I'm writing these lines (April 9th, 2019)  the CUDA version is upgraded on Colab and you should download another version of the cuDNN in the next step. Now is release 10.0 and we are using cuDNN (cudnn-10.0-linux-x64-v7.5.0.56.tgz) accordingly_

In [ ]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## STEP 3. Install cuDNN according to the current CUDA version
Now, you need to download cuDNN from Nvidia web site. You'll need to sign up on the site.

* Download cuDNN from [Nvidia website](https://developer.nvidia.com/cudnn)

  * Right now, because we have _**CUDA 10.0**_ preinstalled in Colab runtime, you need download [cuDNN v7.5.0.56 for CUDA v10.0](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz) - the file is cudnn-10.0-linux-x64-v7.5.0.56.tgz

* On your local computer, create a folder named _**cuDNN**_ in your local folder _**darknet**_. Copy the _**tgz**_ file there



In [10]:
# Verificar se a GPU está ativada no ambiente do Colab
# Isso garante que o treinamento será acelerado com CUDA/cuDNN
!nvidia-smi

# Verificar se a biblioteca cuDNN está instalada e acessível
# O arquivo 'cudnn_version.h' contém as macros com a versão instalada
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2


/bin/bash: line 1: nvidia-smi: command not found
#define CUDNN_MAJOR 9
#define CUDNN_MINOR 2
#define CUDNN_PATCHLEVEL 1
--
#define CUDNN_VERSION (CUDNN_MAJOR * 10000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


## STEP 4. Installing Darknet
Great!! We have all the necessary to start working with Darknet.

This notebook works with a slightly modified version of darknet, which is based on the [AlexeyAB Darknet repo](https://github.com/AlexeyAB/darknet/)
The changes applied to the original repo are:
* Allowing to use spaces on the darknet configuration files _**obj.data**_. Necessary to work with Google Drive directly.
* Removing some logs on every epoch. The original repo write more logs than the Colab notebook can sync. This creates a long queue during the training. This version only shows the results after every iteration.

You can take a look to the code at the [github repo](https://github.com/kriyeng/darknet/)

> _**TRICK**: Because we want to run the notebook fast every time we will compile darknet only the first time we run this notebook. Then, we'll save the compiled version to your drive. For the future executions we'll copy the compiled one instead of compiling again._


**Here comes our first trick to speed up the runtime load time**

* The first time we will:
  * Clone and compile the darknet project.
  * Copy the compiled version to our Google Drive Floder
  
* The next times, instead of compiling it again on every runtime load, we'll copy the compiled version to our VM machine!

When compiling the first time, your output last line has to be something like this:

`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)`

## STEP 4-A. Cloning and compiling Darkent. ONLY NEEDS TO BE RUN ON THE FIRST EXECUTION!!
In this step we'll clone the darkent repo and compile it.
* Clone Repo
* Compile Darknet
* Copy compiled version to Drive

When compiling ends, your output last line has to be something like this:

_`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)_`

In [11]:
# Atualizar a lista de pacotes disponíveis no sistema
!apt-get update

# Instalar o pacote de desenvolvimento do OpenCV
# Necessário para compilar o Darknet com suporte a leitura de imagens, vídeos e visualização
!apt-get install -y libopencv-dev


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,008 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,802 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubunt

In [12]:
# Clonar o repositório oficial do Darknet modificado por AlexeyAB
!git clone https://github.com/AlexeyAB/darknet.git darknet_base

# Mudar para o diretório recém-clonado
%cd darknet_base

# Verificar os arquivos dentro da pasta 'darknet'
!ls

Cloning into 'darknet_base'...
remote: Enumerating objects: 15909, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15909 (delta 2), reused 2 (delta 2), pack-reused 15903 (from 2)
Receiving objects: 100% (15909/15909), 14.46 MiB | 19.15 MiB/s, done.
Resolving deltas: 100% (10705/10705), done.
/content/darknet_base
3rdparty		data		       net_cam_v3.sh
build			docker-compose.yml     net_cam_v4.sh
build.ps1		Dockerfile.cpu	       package.xml
cfg			Dockerfile.gpu	       README.md
cmake			image_yolov3.sh        scripts
CMakeLists.txt		image_yolov4.sh        src
DarknetConfig.cmake.in	include		       vcpkg.json
darknet_images.py	json_mjpeg_streams.sh  video_yolov3.sh
darknet.py		LICENSE		       video_yolov4.sh
darknet_video.py	Makefile


In [22]:
# Copiar o Makefile e as pastas 'src' e 'include' do repositório clonado para sua pasta principal 'darknet'
!cp /content/darknet_base/Makefile /content/darknet/
!cp -r /content/darknet_base/src /content/darknet/
!cp -r /content/darknet_base/include /content/darknet/
#Copiar a pasta '3rdparty' que contém 'stb_image.h'
!cp -r /content/darknet_base/3rdparty /content/darknet/


In [23]:
# 🔧 Editar automaticamente o Makefile para ativar os recursos essenciais
!sed -i 's/GPU=0/GPU=1/' /content/darknet/Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' /content/darknet/Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' /content/darknet/Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' /content/darknet/Makefile

In [24]:
%cd /content/darknet

# Remover a linha que tenta aplicar chmod em arquivos inexistentes
!sed -i '/chmod +x \*.sh/d' /content/darknet/Makefile

/content/darknet


**Nessa parte por limite de uso de GPU/Cuda mudei na configuração o uso para CPU**

In [29]:
# Ajustar o Makefile para CPU
!sed -i 's/GPU=1/GPU=0/' /content/darknet/Makefile
!sed -i 's/CUDNN=1/CUDNN=0/' /content/darknet/Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' /content/darknet/Makefile

In [30]:
# Limpar compilações anteriores
!make clean

# Compilar o Darknet com suporte a GPU, cuDNN, OpenCV e float16
!make -C /content/darknet


rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/representation_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/rnn.o ./obj/rnn_vid.o ./obj/crnn_layer.o ./obj/demo.o ./obj/tag.o ./obj/cifar.o ./obj/go.o ./obj/batchnorm_layer.o ./obj/art.o ./obj/region_layer.o ./obj

In [31]:
# Testar se o binário foi compilado corretamente
!/content/darknet/darknet


usage: /content/darknet/darknet <function>


In [32]:
!ls -l /content/darknet/darknet

-rwxr-xr-x 1 root root 1570272 Sep 21 00:38 /content/darknet/darknet


## STEP 4-B. Copying the compiled version of Darknet from Drive. UNCOMMENT AFTER FIRST EXECUTION
Copy the darknet compiled version from drive to the VM.
* Make the local darknet folder
* Copy the darknet file
* Set execution permissions

## STEP 5. Runtime configuration finished!
Let's chek it out!

If you are running this notebook for the first time, you can run the following cells in order to check if everything goes as expected!




## Some Utils
Let's add some utils that maybe can be useful.

These utils are:
* imgShow() - Will help us to show an image in the remote VM
* download() - Will allow you to get some file from your notebook in case you need to
* upload() - You can upload files to your current folder on the remote VM.

Thanks to [Ivan Goncharov](https://twitter.com/Ivangrov) for these helpers!

In [33]:
#baixar arquivos
%matplotlib inline
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
  return list(uploaded.keys())

def download(path):
  from google.colab import files
  files.download(path)

In [37]:
# Baixar os pesos corretos do YOLOv3 (~248 MB)
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights


--2025-09-21 01:16:50--  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov3.weights
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/75388965/e42c2500-9016-11ea-92ba-11df9f79f31b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-21T02%3A06%3A39Z&rscd=attachment%3B+filename%3Dyolov3.weights&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-21T01%3A05%3A52Z&ske=2025-09-21T02%3A06%3A39Z&sks=b&skv=2018-11-09&sig=tsZmBdHRFmw9prW4V7WEWwL5GsNKYArm8yBP0%2BtK13U%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODQxNzcxMSwibmJmIjoxNzU4NDE3NDExLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZ

**NOTE:** The following test only will work when the darknet is compiled in the runtime. This demo uses some data from the original darknet folders. For your Object detection projects, you'll have these necessary files on your local folder.

In [41]:
# Executar detecção com YOLOv3 no modo CPU
!/content/darknet/darknet detector test /content/darknet/val2017 /content/darknet/cfg/yolov3.cfg /content/darknet/yolov3.weights /content/darknet/val2017/000000000139.jpg -dont_show

# Verificar se a imagem foi gerada
!ls -l /content/darknet/predictions.jpg

 GPU isn't used 
 OpenCV version: 4.5.4
names: Using default 'data/names.list'
Couldn't open file: data/names.list
ls: cannot access '/content/darknet/predictions.jpg': No such file or directory


In [38]:
# Execute darknet usando o modelo YOLOv3 com pesos pré-treinados para detectar objetos em 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Mostrar o resultado usando o auxiliar imgShow()
imShow('predictions.jpg')

 GPU isn't used 
 OpenCV version: 4.5.4
Couldn't open file: cfg/coco.data


AttributeError: 'NoneType' object has no attribute 'shape'

## If you can see the same picture as the one below, congratulations!! At this point you have Darknet configured and working!

![person.jpg](http://blog.ibanyez.info/download/B20190409T000000064.png)

# PART 2. Training YOLO

 > _**TRICK:** Every time you want to run all your cells automatically you can go to the **> Menu > Runtime > run all**. Maybe you don't want to execute the entire notebook. You can write the following cell where you want to stop the process and uncoment the **`assert False`** command. This will throw an error and will stop to run more cells. Thanks to: [This thread](https://groups.google.com/forum/#!topic/jupyter/ELftSFSiedQ)_




In [ ]:
# don't go beyond here with Run All
#assert False

## PART 2. STEP 0. Preparing your data and configuration files

Before going further let's take a look at what configuration files you need to have in your local drive _`darknet`_

![Yolov3 configuration files cheat sheet.jpg](http://blog.ibanyez.info/download/B20190410T000000072.png)

You can download the cheat sheet [here](http://blog.ibanyez.info/download/B20190410T000000072.png)

If you need deeper explanations on how to prepare your data sets, annotation and deep learning, visit [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](post link)


> **TRICK:** You have to be carefully configuring paths on your config files. _*obj.data*_ file needs to have spaces on the path escaped with _**\**_. Like this: **_/content/gdrive/My\ Drive/darknet/train.txt_**. But, in files **_train.txt_** and **_test.txt_** does not!




## PART 2. STEP 1. Loading files to VM local drive
The network speed between Google Drive and Colab VM can have an impact on your training speed accessing to your dataset images.

You can have 3 possible approaches depending on the size of your dataset.

> _**NOTE:** This step is not necessary for all the configuration files and weights. They can be accessed directly from Google Drive without considerable performance issues. **This step is only for the dataset images and annotations**_

* **Option 1** - You can try  to use directly the files from Google Drive _`img/`_ folder. Depending on your dataset maybe this can be more than good.
* **Option 2** - Before start training copy your dataset from Google Drive to the local VM filesystem. Maybe can be a good practice to copy as one single tar file and decompress in your VM local _`img/`_ folder
* **Option 3** - If your dataset is quite big, maybe you can upload to a git repository and clone from here. Usually transfer time between are much better. If you have to decide I have the feeling that bitbucket have better speed transfer times than github, but please, don't take this as confirmed, **I haven't done specific tests on that, I could be wrong!**




#### PART 2. STEP 1 - Option 1. Using files from Google Drive directly.
You don't have to do anything here. Your **_train.txt_** should have the correct path:
* **/content/grdive/My Drive/darknet/img/image001.jpg**. As said before, don't use escaped white space for the paths on _**train.txt**_ and _**test.txt**_

#### PART 2. STEP 1 - Option 2A. Copying files from Google Drive to VM local filesystem.
Execute the follow cell to copy your files

In [ ]:
# Copia dos arquivos do Google Drive para o sistema de arquivos local da VM
!cp -r "/content/gdrive/My Drive/darknet/dataset_coco_cachorros" /content/darknet_local/
!cp -r "/content/gdrive/My Drive/darknet/dataset_coco_gatos" /content/darknet_local/


In [ ]:
!ls "/content/gdrive/My Drive/darknet"


annotations  bin  cuDNN  dataset_coco_cachorros  dataset_coco_gatos  val2017


#### PART 2. STEP 1 - Option 2B. Copying files zipped from Google Drive to VM local filesystem and unzip locally.
Execute the follow cell to copy your files and uncompress.
You can use _*!ls*_ command to esnure what's the correct path you have to configure in your _*train.txt*_ to correctly access to your dataset images


In [ ]:
# Copia seu arquivo compactado
#!cp -r "/content/gdrive/My Drive/darknet/img/img.tgz" ./img

# Descompacte o arquivo compactado
#!tar -xzvf ./img/img.tgz

#### PART 2. STEP 1 - Option 3. Clone your image dataset from a git repo. Seems the fastest one.
Execute the follow cell to clone your dataset repo to VM local filesystem


In [ ]:
# Clona o Git diretamente para a pasta ./img
#!git clone https://[your-repository] ./img

# Verifique o resultado - Descomente quando você verificar a velocidade para executar mais execuções
#!ls -la ./img

## PART 3. Finally, Train your model!

When you execute the following command, your model will start training.

You will have a log line per epoch. On each iteration you will see how your training is going.

> **TRICK: Darknet copies a backup of your trained weights every 100 iterations. As magic, this file will be synced to your local drive on your computer, as well as the backups darknet do every 1000 iterations, saving it on a separate file.**





In [ ]:
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/darknet53.conv.74" -dont_show

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
Couldn't open file: /content/gdrive/My Drive/darknet/obj.data


In [ ]:
!ls "/content/gdrive/My Drive/darknet/obj.data"


ls: cannot access '/content/gdrive/My Drive/darknet/obj.data': No such file or directory


In [ ]:
#tive retorno que o Darknet disse que o arquivo obj.data simplesmente não existe no caminho passado original
#tentar contornar para iniciar o treinamento com 4 passos


In [ ]:
#Etapa 1-
#criar o arquivo obj.data
obj_data = """classes = 2
train = /content/darknet_local/train.txt
valid = /content/darknet_local/test.txt
names = /content/darknet_local/obj.names
backup = /content/darknet_local/backup/
"""

with open("/content/darknet_local/obj.data", "w") as f:
    f.write(obj_data)


In [ ]:
#Etapa 2-
#criar os arquivos auxiliares
with open("/content/darknet_local/obj.names", "w") as f:
    f.write("cat\n")
    f.write("dog\n")

In [ ]:
#Etapa 2.2
#gerar automaticamente os caminhos das imagens copiadas para a Vm

import glob

train_images = glob.glob("/content/darknet_local/dataset_coco_gatos/*.jpg") + \
               glob.glob("/content/darknet_local/dataset_coco_cachorros/*.jpg")

with open("/content/darknet_local/train.txt", "w") as f:
    for path in train_images:
        f.write(path + "\n")

# Se quiser usar parte como validação:
with open("/content/darknet_local/test.txt", "w") as f:
    for path in train_images[:20]:  # primeiros 20 como exemplo
        f.write(path + "\n")


In [ ]:
#Etapa 3-
#criar pasta de backup

!mkdir -p /content/darknet_local/backup


In [ ]:
#Etapa 4-
#executar o treinamento=
!./darknet detector train /content/darknet_local/obj.data /content/gdrive/My\ Drive/darknet/yolov3.cfg /content/gdrive/My\ Drive/darknet/darknet53.conv.74 -dont_show


 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov3
Couldn't open file: /content/gdrive/My Drive/darknet/yolov3.cfg


In [ ]:
#copia/cria arquivo "yolov3.cfg" para o google drive
!cp cfg/yolov3.cfg /content/gdrive/My\ Drive/darknet/yolov3.cfg

In [ ]:
#tentando rodar treinamento novamente
!./darknet detector train /content/darknet_local/obj.data /content/gdrive/My\ Drive/darknet/yolov3.cfg /content/gdrive/My\ Drive/darknet/darknet53.conv.74 -dont_show


 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov3
mini_batch = 1, batch = 1, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv  

In [ ]:
#Baixar os arquivos de pesos pre-treinados
!wget https://pjreddie.com/media/files/darknet53.conv.74 -P "/content/gdrive/My Drive/darknet/"

--2025-09-08 06:51:17--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 104.21.88.156, 172.67.185.199, 2606:4700:3030::ac43:b9c7, ...
Connecting to pjreddie.com (pjreddie.com)|104.21.88.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/gdrive/My Drive/darknet/darknet53.conv.74’

darknet53.conv.74       [ <=>                ]   8.88K  --.-KB/s    in 0.008s  

2025-09-08 06:51:17 (1.03 MB/s) - ‘/content/gdrive/My Drive/darknet/darknet53.conv.74’ saved [9093]



In [ ]:
#reexecuta o treinamento
!./darknet detector train /content/darknet_local/obj.data /content/gdrive/My\ Drive/darknet/yolov3.cfg /content/gdrive/My\ Drive/darknet/darknet53.conv.74 -dont_show

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov3
mini_batch = 2, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv 

## PERFORMANCE TIPS & TRICKS

* **Speed up load times of the runtime:** When everything is checked that works, you can remove cells or comment unnecessary lines of code to make your loading time lower on every run.

* **How to keep your notebook alive for more time?:** Keep you browser with your notebook open. If you close your browser, your notebook will reach the iddle time and will be removed from Colab cloud Service. (90 minutes)
  
* **Re-run your training after reaching the limitation time for Colab runtimes (12 hours):**
  * Open a new notebook or reconnect the current one.
  * Comment the cell above and uncomment the cell below.
  * In your local computer, copy the file **backup/yolov3_last.weights** to your local computer **weights/** folder.
  * Execute Run all in the **> menu > Runtime > Run All**
  * _The copy step is not absolutely necessary, but I like to keep a copy of the last training session and not overwrite this file on next trainings._

In [ ]:
# Start training at the point where the last runtime finished
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show

## TROUBLESHOOTING
The main problems you can face if your model throw an error is:

* Images or files not found. Check the **Yolov3 cheat sheet** image above and check that everything is ok.
* If have you configured wrongly your **filters** and **classes** in the **yolov3.cfg**. Check the **Yolov3 cheet sheet** above.
* You can face some out of memory or library errors mainly for the lack of some the pre-requisits. In this case, check the versions of the current libraries installed on your Colab VM. You can find more information in the first steps of this notebook.
* **Batch** and **subdivisions** parameters on your **yolov3.cfg** can affect to the memory allocation as well. Refer to the original repo [Here]() for further details.

### TROUBLESHOOTING UPDATE
Be careful if you are preparing your files on Windows. If you use **CRLF** on your files instead of **LF** You can have problems opening the files correctly. - Thanks to [Satya Shetty](https://twitter.com/satyashetty) for sharing this issue!


## About me

You can find the original post with more explanations about this notebook at [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](post link)

I'm David Ibañez from Barcelona. Feel free to get in touch!

* You can visit my blog at [Dev-ibanyez.info](http://blog.ibanyex.info)
* You can get in touch with me on [Twitter](https://twitter.com/dav_ibanez)
* You can get in touch or contribute to this notebook at [Github](https://github.com/kriyeng/yolo-on-colab-notebook/)
* You can comment on the [dev.to post about this notebook ](PENDING)

Thanks for you having read this notebook! :clap: :clap: :clap:

## SOURCES


#### Other sources
* YOLO original web site [Joseph Redmon Page](https://pjreddie.com/darknet/yolo/)
* AlexeyAB darknet repo [github](https://github.com/AlexeyAB/darknet/)
* The Ivan Goncharov [notebook](https://github.com/ivangrov/YOLOv3-GoogleColab/blob/master/YOLOv3_GoogleColab.ipynb) inspired me to try Google Colab and end up creating this notebook.